In [2]:
from cnf import CNFFormula, Clause
from typing import Optional, Dict, List

In [72]:
class CDCL:
    def __init__(self, cnf: CNFFormula):
        self.cnf = cnf
        self.assignments: Dict[int, bool] = {}
        self.trail: List[int] = [] # List of (literal, decision_level, antecedent_clause)
        self.decision_level = 0
        

    def literal_status(self, literal: int) -> Optional[bool]:
        var = abs(literal)
        if var not in self.assignments:
            return None
        value = self.assignments[var]
        return value if literal > 0 else not value
    
    def clause_status(self, clause: Clause) -> Optional[bool]:
        has_unassigned = False
        for lit in clause.literals:
            status = self.literal_status(lit)
            if status is True:
                return True
            if status is None:
                has_unassigned = True
        
        if has_unassigned:
            return None
        return False
    
    def find_unit_clauses(self):
        '''
        Clause is unit iff clause is NOT satisfied, exactly 1 literal unassigned and all other literals False
        
        Returns list of (unit_literal, clause) pairs
        '''
        units = []
        for clause in self.cnf.clauses:
            unassigned_lit = None
            all_false = True
            
            for lit in clause.literals:
                status = self.literal_status(lit)
                if status == True: # Not unit clause (Clause already True)
                    all_false = False
                    break
                if status is None:
                    if unassigned_lit == None:
                        unassigned_lit = lit
                    else: # Not unit clause (Multiple Unassigned literals)
                        all_false = False
                        break
            if all_false and unassigned_lit is not None:
                units.append((unassigned_lit, clause))
        return units
    
    def assign(self, literal: int, antecedent: Optional[Clause] = None):
        '''
        Assigns a literal at the current decision level.
        Returns False if a conflict arises, else True
        '''
        var = abs(literal)
        value = literal > 0
        if var in self.assignments:
            if self.assignments[var] != value:
                return False  # Conflict
            return True  # Already assigned consistently
        
        self.assignments[var] = value
        self.trail.append((literal, self.decision_level, antecedent))
        return True
    
    def unit_propagate(self):
        '''
        Performs Unit propagation
        Returns a conflict clause if conflict arises, else None
        '''
        
        while True:
            units = self.find_unit_clauses()
            if not units:
                return None
            
            for lit, clause in units:
                status = self.clause_status(clause)
                if status == False:
                    return clause  # Conflict detected
                
                if status is None:
                    check = self.assign(lit, clause)
                    if not check:
                        return clause  # Conflict detected
        return None 
    
    def decide(self, literal: int):
        '''
        Makes a decision assignment at a new decision level
        '''
        self.decision_level += 1
        check = self.assign(literal)
        assert check, "Conflict on decision assignment"
        
    def find_conflict(self) -> Optional[Clause]:
        '''
        Checks for conflicts in the current assignments
        Returns the first conflicting clause found, else None
        '''
        for clause in self.cnf.clauses:
            status = self.clause_status(clause)
            if status == False:
                return clause
        return None
    
    def backjump(self, level: int):
        '''
        Backjumps to the specified decision level
        Removes all assignments made at higher decision levels
        '''
        print(f'Backtracking from level {self.decision_level} to level {level}')
        
        new_trail = []
        new_assignments = {}
        print(self.trail[0])
        for lit, dl, antecedent in self.trail:
            if dl <= level:
                new_trail.append((lit, dl, antecedent))
                var = abs(lit)
                value = lit > 0
                new_assignments[var] = value

        self.trail = new_trail
        self.assignments = new_assignments
        self.decision_level = level
        
    def trail_level(self, var: int) -> int:
        '''
        Returns the decision level at which the variable was assigned
        '''
        for lit, dl, _ in reversed(self.trail):
            if abs(lit) == var:
                return dl
        return -1  # Variable not assigned

    def resolve(self, clause1: List[int], clause2: List[int], literal: int) -> List[int]:
        '''
        Resolves two clauses on the given pivot literal
        Returns the resulting clause as a list of literals
        '''
        new_clause = []

        for l in clause1:
            if l != literal:
                new_clause.append(l)

        for l in clause2:
            if l != -literal and l not in new_clause:
                new_clause.append(l)
        print('ausfousa ', new_clause)
        return new_clause
    
    def analyze_conflict(self, conflict_clause: Clause):
        '''
        Performs conflict analysis and learns a new clause and backjump level
        '''
        
        learned = conflict_clause.literals.copy()
        while True:
            level_count = sum(1 for lit in learned if self.trail_level(abs(lit)) == self.decision_level)
            if level_count <= 1:
                break
            print('HERE')
            for lit, dl, antecedent in reversed(self.trail):
                print('ANALYZE LOOP ', learned, lit, dl)
                if abs(lit) in [abs(l) for l in learned] and dl == self.decision_level:
                    if antecedent is not None:
                        print('TEST RESOLVE ENTER')
                        learned = self.resolve(learned, antecedent.literals, abs(lit))
                    break
        backjump_level = max((self.trail_level(abs(lit)) for lit in learned if self.trail_level(abs(lit)) < self.decision_level), default=0)
        return learned, backjump_level
    def solve(self) -> bool:
        '''
        CDCL Solver main loop
        Returns True if Satisfiable, False if unsatisfiable
        '''
        
        while True: 
            # Unit Propagation
            conflict = self.unit_propagate()
            if conflict is not None:
                if self.decision_level == 0:
                    self.cnf.satisfiable = False
                    return False  # Unsatisfiable
                learned_clause, backjump_level = self.analyze_conflict(conflict)
                self.backjump(backjump_level)
                self.cnf.add_clause(Clause(learned_clause))
                continue
                
            if len(self.assignments) == self.cnf.num_vars:
                # Sanity Check
                for clause in self.cnf.clauses:
                    status = self.clause_status(clause)
                    assert status == True, "All clauses should be satisfied"
                self.cnf.satisfiable = True
                return True  # Satisfiable
            
            for var in range(1, self.cnf.num_vars + 1):
                if var not in self.assignments:
                    self.decide(var)
                    break


In [73]:
# --- Test CDCL Solver ---

# Example CNF: (x1 OR x2) AND (¬x1 OR x3) AND (¬x2 OR ¬x3)
cnf = CNFFormula(num_vars=3)
cnf.add_clause(Clause([1, 2]))
cnf.add_clause(Clause([-1, 3]))
cnf.add_clause(Clause([-2, -3]))

solver = CDCL(cnf)

result = solver.solve()

print("\n--- RESULTS ---")
print("Satisfiable:", result)
print("Assignments:", solver.assignments)
print("Trail:")
for t in solver.trail:
    print(t)
print("\nClauses after learning:")
for c in cnf.clauses:
    print(c)



--- RESULTS ---
Satisfiable: True
Assignments: {1: True, 3: True, 2: False}
Trail:
(1, 1, None)
(3, 1, Clause([-1, 3]))
(-2, 1, Clause([-2, -3]))

Clauses after learning:
Clause([1, 2])
Clause([-1, 3])
Clause([-2, -3])


Learned clause: [-1, -2]
Backjump level: 1
